# Data Preprocessing

Source: https://www.kaggle.com/donjoeml/energy-consumption-and-generation-in-the-globe


In [1]:
# Import dependencies
import io
import re
from itertools import groupby
from functools import reduce

import pandas as pd
import numpy as np
from IPython.display import display

## Import & inspect

In [2]:
# Setup imports
file_paths = {
    'power_plants': '../data/global_power_plant_database_last.csv', 
    'primary_consumption': '../data/Primary-energy-consumption-from-fossilfuels-nuclear-renewables.csv', 
    'shared_production': '../data/share-elec-produc-by-source.csv', 
    'shared_consumption': '../data/share-energy-consum-by-source.csv'
}

In [3]:
# Read in datasets
dfs = {}

for name, path in file_paths.items():
    dfs[name] = pd.read_csv(path)

/var/folders/dl/js17w9rx0xg2plhj4chp19pw0000gn/T/ipykernel_26713/3848680433.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[name] = pd.read_csv(path)


In [4]:
# Display head of every dataframe
for name, df in dfs.items():
    print(name.upper())
    display(df.head())

POWER_PLANTS


,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.0,32.322,65.1190,Hydro,NaN,NaN,...,123.77,162.90,97.39,137.76,119.50,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
1,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.0,31.670,65.7950,Solar,NaN,NaN,...,18.43,17.48,18.25,17.70,18.29,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
2,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.0,31.623,65.7920,Solar,NaN,NaN,...,18.64,17.58,19.10,17.62,18.72,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.0,34.556,69.4787,Hydro,NaN,NaN,...,225.06,203.55,146.90,230.18,174.91,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.0,34.641,69.7170,Hydro,NaN,NaN,...,406.16,357.22,270.99,395.38,350.80,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1


PRIMARY_CONSUMPTION


,Entity,Code,Year,Fossil fuels (% sub energy),Renewables (% sub energy),Nuclear (% sub energy)
0,Africa,NaN,1965,94.463,5.537,0.0
1,Africa,NaN,1966,94.127,5.873,0.0
2,Africa,NaN,1967,93.920,6.080,0.0
3,Africa,NaN,1968,93.257,6.743,0.0
4,Africa,NaN,1969,92.317,7.683,0.0


SHARED_PRODUCTION


,Entity,Code,Year,Coal (% electricity),Gas (% electricity),Hydro (% electricity),Solar (% electricity),Wind (% electricity),Oil (% electricity),Nuclear (% electricity),Other renewables (% electricity)
0,Afghanistan,AFG,2000,0.0,33.190578,66.809422,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,AFG,2001,0.0,15.878378,84.121622,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,AFG,2002,0.0,19.213974,80.786026,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,AFG,2003,0.0,32.907348,67.092652,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,AFG,2004,0.0,36.800895,63.199105,0.0,0.0,0.0,0.0,0.0


SHARED_CONSUMPTION


,Entity,Code,Year,Oil (% sub energy),Coal (% sub energy),Solar (% sub energy),Nuclear (% sub energy),Hydro (% sub energy),Wind (% sub energy),Gas (% sub energy),Other renewables (% sub energy)
0,Africa,NaN,1965,46.720,46.375,0.0,0.0,5.537,0.0,1.368,0.0
1,Africa,NaN,1966,48.626,44.047,0.0,0.0,5.873,0.0,1.454,0.0
2,Africa,NaN,1967,47.838,44.655,0.0,0.0,6.080,0.0,1.426,0.0
3,Africa,NaN,1968,47.859,44.029,0.0,0.0,6.743,0.0,1.370,0.0
4,Africa,NaN,1969,47.373,43.381,0.0,0.0,7.683,0.0,1.563,0.0


In [5]:
# Display columns, null counts, and coverage

num_rows_pattern = re.compile(r':\s([^\s]+)')
column_num_pattern = re.compile(r'^\s*[^\s]+')
extract_info_pattern = re.compile(r'\s{2,}')

def extract_info(df, info_str):

    def transform_str(info_line):
        #  Remove "non-null" and column number
        info_line = re.sub(column_num_pattern, '', info_line.replace('non-null', ''))

        # Extract remaining strings
        return re.split(extract_info_pattern, info_line.strip())


    rows_match = re.findall(num_rows_pattern, info_str.split('\n')[1])
    num_rows = int(rows_match[0])

    # Build dataframe with info data    
    info_df = pd.DataFrame([transform_str(s) for s in info_str.split('\n')[5:-3]], 
                                columns=['column', 'non_null', 'dtype'])

    # Calculate coverage percentage
    info_df['non_null'] = info_df['non_null'].astype(int)
    info_df['coverage'] = info_df['non_null'] / num_rows

    # Include number of unique values
    info_df['nunique'] = df.nunique().values

    info_df = info_df[['column', 'non_null', 'coverage', 'nunique', 'dtype']].set_index('column')

    return info_df


for name, df in dfs.items():
    print(name.upper())

    buffer = io.StringIO()
    df.info(buf=buffer)
    info_str = buffer.getvalue()

    display(extract_info(df, info_str))

POWER_PLANTS


,non_null,coverage,nunique,dtype
column,,,,
country,34936,1.000000,167,object
country_long,34936,1.000000,167,object
name,34936,1.000000,34528,object
gppd_idnr,34936,1.000000,34936,object
capacity_mw,34936,1.000000,5611,float64
latitude,34936,1.000000,31779,float64
longitude,34936,1.000000,33036,float64
primary_fuel,34936,1.000000,15,object
other_fuel1,1944,0.055645,12,object


PRIMARY_CONSUMPTION


,non_null,coverage,nunique,dtype
column,,,,
Entity,4284,1.000000,83,object
Code,4119,0.961485,80,object
Year,4284,1.000000,55,int64
Fossil fuels (% sub energy),4284,1.000000,3551,float64
Renewables (% sub energy),4284,1.000000,3422,float64
Nuclear (% sub energy),4284,1.000000,1454,float64


SHARED_PRODUCTION


,non_null,coverage,nunique,dtype
column,,,,
Entity,6190,1.000000,235,object
Code,5423,0.876090,211,object
Year,6190,1.000000,36,int64
Coal (% electricity),5170,0.835218,2309,float64
Gas (% electricity),5170,0.835218,3069,float64
Hydro (% electricity),6190,1.000000,4793,float64
Solar (% electricity),6190,1.000000,2844,float64
Wind (% electricity),6190,1.000000,2729,float64
Oil (% electricity),5170,0.835218,3943,float64


SHARED_CONSUMPTION


,non_null,coverage,nunique,dtype
column,,,,
Entity,4284,1.000000,83,object
Code,4119,0.961485,80,object
Year,4284,1.000000,55,int64
Oil (% sub energy),4284,1.000000,4118,float64
Coal (% sub energy),4284,1.000000,3569,float64
Solar (% sub energy),4284,1.000000,434,float64
Nuclear (% sub energy),4284,1.000000,1454,float64
Hydro (% sub energy),4284,1.000000,3206,float64
Wind (% sub energy),4284,1.000000,789,float64


In [6]:
consumption_df = dfs['shared_consumption']
consumption_df.head(10)

,Entity,Code,Year,Oil (% sub energy),Coal (% sub energy),Solar (% sub energy),Nuclear (% sub energy),Hydro (% sub energy),Wind (% sub energy),Gas (% sub energy),Other renewables (% sub energy)
0,Africa,NaN,1965,46.720,46.375,0.0,0.0,5.537,0.0,1.368,0.000
1,Africa,NaN,1966,48.626,44.047,0.0,0.0,5.873,0.0,1.454,0.000
2,Africa,NaN,1967,47.838,44.655,0.0,0.0,6.080,0.0,1.426,0.000
3,Africa,NaN,1968,47.859,44.029,0.0,0.0,6.743,0.0,1.370,0.000
4,Africa,NaN,1969,47.373,43.381,0.0,0.0,7.683,0.0,1.563,0.000
5,Africa,NaN,1970,48.014,41.398,0.0,0.0,8.807,0.0,1.781,0.000
6,Africa,NaN,1971,49.942,40.437,0.0,0.0,7.629,0.0,1.943,0.048
7,Africa,NaN,1972,51.049,38.253,0.0,0.0,8.196,0.0,2.456,0.046
8,Africa,NaN,1973,50.710,37.603,0.0,0.0,8.007,0.0,3.637,0.043
9,Africa,NaN,1974,50.013,37.345,0.0,0.0,8.693,0.0,3.906,0.043


In [7]:
# Inspect column stats
consumption_df.describe()

,Year,Oil (% sub energy),Coal (% sub energy),Solar (% sub energy),Nuclear (% sub energy),Hydro (% sub energy),Wind (% sub energy),Gas (% sub energy),Other renewables (% sub energy)
count,4284.000000,4284.00000,4284.000000,4284.000000,4284.000000,4284.000000,4284.000000,4284.000000,4284.000000
mean,1993.113912,48.48012,17.763976,0.079402,3.126016,8.624511,0.345406,20.811585,0.626893
std,15.678077,20.78287,19.723911,0.336582,6.639930,12.279163,1.337962,20.448799,1.867764
min,1965.000000,5.05300,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1980.000000,34.16675,2.005750,0.000000,0.000000,0.521750,0.000000,3.839750,0.000000
50%,1994.000000,45.07950,11.035000,0.000000,0.000000,4.084000,0.000000,15.838000,0.039000
75%,2007.000000,62.36550,26.766250,0.001000,2.121750,10.854250,0.020000,30.498750,0.442250
max,2019.000000,100.00000,89.158000,3.593000,41.707000,71.055000,20.660000,90.744000,24.594000


In [8]:
production_df = dfs['shared_production']
production_df.head(10)

,Entity,Code,Year,Coal (% electricity),Gas (% electricity),Hydro (% electricity),Solar (% electricity),Wind (% electricity),Oil (% electricity),Nuclear (% electricity),Other renewables (% electricity)
0,Afghanistan,AFG,2000,0.0,33.190578,66.809422,0.0,0.000000,0.0,0.0,0.0
1,Afghanistan,AFG,2001,0.0,15.878378,84.121622,0.0,0.000000,0.0,0.0,0.0
2,Afghanistan,AFG,2002,0.0,19.213974,80.786026,0.0,0.000000,0.0,0.0,0.0
3,Afghanistan,AFG,2003,0.0,32.907348,67.092652,0.0,0.000000,0.0,0.0,0.0
4,Afghanistan,AFG,2004,0.0,36.800895,63.199105,0.0,0.000000,0.0,0.0,0.0
5,Afghanistan,AFG,2005,0.0,36.830835,63.169165,0.0,0.000000,0.0,0.0,0.0
6,Afghanistan,AFG,2006,0.0,23.894863,76.105137,0.0,0.000000,0.0,0.0,0.0
7,Afghanistan,AFG,2007,0.0,20.930233,79.069767,0.0,0.000000,0.0,0.0,0.0
8,Afghanistan,AFG,2008,0.0,25.549100,74.449527,0.0,0.001374,0.0,0.0,0.0
9,Afghanistan,AFG,2009,0.0,17.268948,82.720392,0.0,0.010660,0.0,0.0,0.0


In [9]:
# Inspect column stats
production_df.describe()

,Year,Coal (% electricity),Gas (% electricity),Hydro (% electricity),Solar (% electricity),Wind (% electricity),Oil (% electricity),Nuclear (% electricity),Other renewables (% electricity)
count,6190.000000,5170.000000,5170.000000,6190.000000,6190.000000,6190.000000,5170.000000,6190.000000,6155.000000
mean,2005.501131,17.431687,23.251049,26.213096,0.406232,1.308663,34.659474,5.248678,1.969872
std,9.321596,39.898895,37.708235,30.950684,1.456500,5.890959,40.407903,13.252240,4.691999
min,1985.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2000.000000,0.000000,0.000000,0.145994,0.000000,0.000000,1.680045,0.000000,0.000000
50%,2007.000000,0.000000,8.004601,12.178127,0.000000,0.000000,12.568226,0.000000,0.065476
75%,2013.000000,24.295534,32.603113,47.546039,0.057878,0.139802,68.453354,0.000000,1.558274
max,2020.000000,523.074933,310.456745,100.000000,25.711136,100.000000,267.854610,86.919532,49.047163


In [10]:
us_production = production_df[production_df['Entity'] == ('United States')]
us_production.describe()

,Year,Coal (% electricity),Gas (% electricity),Hydro (% electricity),Solar (% electricity),Wind (% electricity),Oil (% electricity),Nuclear (% electricity),Other renewables (% electricity)
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,2002.500000,46.318197,20.386052,7.534798,0.379748,1.749257,2.618609,19.135737,1.759312
std,10.535654,10.161135,8.704877,1.378732,0.827357,2.464041,1.484237,1.081721,0.149973
min,1985.000000,19.107033,9.326943,5.570175,0.000315,0.000030,0.714773,15.199931,1.369621
25%,1993.750000,41.429643,13.324740,6.535805,0.013118,0.089176,1.077101,19.026382,1.700258
50%,2002.500000,50.464482,17.776121,7.079513,0.014498,0.278227,2.739704,19.434266,1.763466
75%,2011.250000,53.176270,25.688709,8.441600,0.059972,3.068027,3.569528,19.683156,1.860386
max,2020.000000,56.841540,40.229594,11.097250,3.274951,8.309780,5.600774,20.575318,1.973438


In [11]:
# Inspect country/location values for consumption and production
production_countries = production_df['Entity']
consumption_countries = consumption_df['Entity']

print('production', len(production_countries))
print('consumption', len(consumption_countries))

production 6190
consumption 4284


In [12]:
# Get shared countries
country_overlap = pd.Series(list(set(production_countries).intersection(set(consumption_countries))))
country_overlap

0     United Arab Emirates
1                    Chile
2                  Ireland
3                   Canada
4                  Morocco
              ...         
78                 Belgium
79                   India
80                 Estonia
81                   China
82                 Germany
Length: 83, dtype: object

In [13]:
# Create fully joined df
product_and_consume_df = pd.merge(production_df, consumption_df, on=['Entity', 'Year'])
product_and_consume_df.head()

,Entity,Code_x,Year,Coal (% electricity),Gas (% electricity),Hydro (% electricity),Solar (% electricity),Wind (% electricity),Oil (% electricity),Nuclear (% electricity),Other renewables (% electricity),Code_y,Oil (% sub energy),Coal (% sub energy),Solar (% sub energy),Nuclear (% sub energy),Hydro (% sub energy),Wind (% sub energy),Gas (% sub energy),Other renewables (% sub energy)
0,Africa,NaN,1985,53.869485,10.783022,19.093587,0.0,0.0,13.242261,2.052236,0.103867,NaN,45.185,35.400,0.0,0.664,6.177,0.0,12.540,0.034
1,Africa,NaN,1986,52.600209,10.495367,18.935040,0.0,0.0,13.766315,3.259047,0.243975,NaN,43.475,35.372,0.0,1.074,6.240,0.0,13.759,0.080
2,Africa,NaN,1987,54.316611,11.269555,17.503940,0.0,0.0,13.802774,2.200460,0.234782,NaN,44.311,35.295,0.0,0.730,5.806,0.0,13.780,0.078
3,Africa,NaN,1988,51.625800,11.517103,18.149086,0.0,0.0,14.430585,3.553388,0.211991,NaN,43.246,35.370,0.0,1.158,5.915,0.0,14.242,0.069
4,Africa,NaN,1989,51.267484,12.046066,18.352067,0.0,0.0,14.132786,3.619461,0.201208,NaN,44.704,33.100,0.0,1.221,6.192,0.0,14.715,0.068


In [14]:
# Filter dataframe to only contain overlapping countries
common_countries = product_and_consume_df[product_and_consume_df['Entity'].isin(country_overlap)]
common_countries.head()

,Entity,Code_x,Year,Coal (% electricity),Gas (% electricity),Hydro (% electricity),Solar (% electricity),Wind (% electricity),Oil (% electricity),Nuclear (% electricity),Other renewables (% electricity),Code_y,Oil (% sub energy),Coal (% sub energy),Solar (% sub energy),Nuclear (% sub energy),Hydro (% sub energy),Wind (% sub energy),Gas (% sub energy),Other renewables (% sub energy)
0,Africa,NaN,1985,53.869485,10.783022,19.093587,0.0,0.0,13.242261,2.052236,0.103867,NaN,45.185,35.400,0.0,0.664,6.177,0.0,12.540,0.034
1,Africa,NaN,1986,52.600209,10.495367,18.935040,0.0,0.0,13.766315,3.259047,0.243975,NaN,43.475,35.372,0.0,1.074,6.240,0.0,13.759,0.080
2,Africa,NaN,1987,54.316611,11.269555,17.503940,0.0,0.0,13.802774,2.200460,0.234782,NaN,44.311,35.295,0.0,0.730,5.806,0.0,13.780,0.078
3,Africa,NaN,1988,51.625800,11.517103,18.149086,0.0,0.0,14.430585,3.553388,0.211991,NaN,43.246,35.370,0.0,1.158,5.915,0.0,14.242,0.069
4,Africa,NaN,1989,51.267484,12.046066,18.352067,0.0,0.0,14.132786,3.619461,0.201208,NaN,44.704,33.100,0.0,1.221,6.192,0.0,14.715,0.068


In [15]:
# Assert merged codes are aligned
common_countries[common_countries['Code_x'].fillna(0) != common_countries['Code_y'].fillna(0)]

,Entity,Code_x,Year,Coal (% electricity),Gas (% electricity),Hydro (% electricity),Solar (% electricity),Wind (% electricity),Oil (% electricity),Nuclear (% electricity),Other renewables (% electricity),Code_y,Oil (% sub energy),Coal (% sub energy),Solar (% sub energy),Nuclear (% sub energy),Hydro (% sub energy),Wind (% sub energy),Gas (% sub energy),Other renewables (% sub energy)


In [16]:
# Can drop one of the code columns
common_countries = common_countries.drop('Code_y', axis=1).rename(columns={'Code_x': 'Code'})
common_countries.head()

,Entity,Code,Year,Coal (% electricity),Gas (% electricity),Hydro (% electricity),Solar (% electricity),Wind (% electricity),Oil (% electricity),Nuclear (% electricity),Other renewables (% electricity),Oil (% sub energy),Coal (% sub energy),Solar (% sub energy),Nuclear (% sub energy),Hydro (% sub energy),Wind (% sub energy),Gas (% sub energy),Other renewables (% sub energy)
0,Africa,NaN,1985,53.869485,10.783022,19.093587,0.0,0.0,13.242261,2.052236,0.103867,45.185,35.400,0.0,0.664,6.177,0.0,12.540,0.034
1,Africa,NaN,1986,52.600209,10.495367,18.935040,0.0,0.0,13.766315,3.259047,0.243975,43.475,35.372,0.0,1.074,6.240,0.0,13.759,0.080
2,Africa,NaN,1987,54.316611,11.269555,17.503940,0.0,0.0,13.802774,2.200460,0.234782,44.311,35.295,0.0,0.730,5.806,0.0,13.780,0.078
3,Africa,NaN,1988,51.625800,11.517103,18.149086,0.0,0.0,14.430585,3.553388,0.211991,43.246,35.370,0.0,1.158,5.915,0.0,14.242,0.069
4,Africa,NaN,1989,51.267484,12.046066,18.352067,0.0,0.0,14.132786,3.619461,0.201208,44.704,33.100,0.0,1.221,6.192,0.0,14.715,0.068


In [17]:
common_countries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2890 entries, 0 to 2889
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Entity                            2890 non-null   object 
 1   Code                              2785 non-null   object 
 2   Year                              2890 non-null   int64  
 3   Coal (% electricity)              2150 non-null   float64
 4   Gas (% electricity)               2150 non-null   float64
 5   Hydro (% electricity)             2890 non-null   float64
 6   Solar (% electricity)             2890 non-null   float64
 7   Wind (% electricity)              2890 non-null   float64
 8   Oil (% electricity)               2150 non-null   float64
 9   Nuclear (% electricity)           2890 non-null   float64
 10  Other renewables (% electricity)  2890 non-null   float64
 11  Oil (% sub energy)                2890 non-null   float64
 12  Coal (

In [18]:
# Ensure only one datapoint per (Entity, Year) by averaging
avg_cols = [c for c in common_countries.columns if c not in ['Entity', 'Code', 'Year']]
common_countries_by_year = common_countries.groupby(['Entity', 'Year']).mean()

In [19]:
common_countries_by_year.head(10)

Coal (% electricity)  Gas (% electricity)  Hydro (% electricity)  \
Entity Year                                                                     
Africa 1985             53.869485            10.783022              19.093587   
       1986             52.600209            10.495367              18.935040   
       1987             54.316611            11.269555              17.503940   
       1988             51.625800            11.517103              18.149086   
       1989             51.267484            12.046066              18.352067   
       1990             51.634581            13.371962              17.952613   
       1991             50.703690            14.034394              18.662111   
       1992             50.429953            15.408962              17.639249   
       1993             51.874893            16.253724              16.710382   
       1994             51.156150            17.669055              16.387169   

             Solar (% electricity)  Wind (% electricity)  Oil (% electricity)  \
Entity Year                                                                     
Africa 1985                    0.0              0.000000            13.242261   
       1986                    0.0              0.000000            13.766315   
       1987                    0.0              0.000000            13.802774   
       1988                    0.0              0.000000            14.430585   
       1989                    0.0              0.000000            14.132786   
       1990                    0.0              0.000000            13.522436   
       1991                    0.0              0.000000            12.985502   
       1992                    0.0              0.000000            13.030478   
       1993                    0.0              0.000590            12.353879   
       1994                    0.0              0.001536            11.360706   

             Nuclear (% electricity)  Other renewables (% electricity)  \
Entity Year                                                              
Africa 1985                 2.052236                          0.103867   
       1986                 3.259047                          0.243975   
       1987                 2.200460                          0.234782   
       1988                 3.553388                          0.211991   
       1989                 3.619461                          0.201208   
       1990                 2.649245                          0.240101   
       1991                 2.813556                          0.242463   
       1992                 2.831533                          0.251280   
       1993                 2.138822                          0.269954   
       1994                 2.758980                          0.233590   

             Oil (% sub energy)  Coal (% sub energy)  Solar (% sub energy)  \
Entity Year                                                                  
Africa 1985              45.185               35.400                   0.0   
       1986              43.475               35.372                   0.0   
       1987              44.311               35.295                   0.0   
       1988              43.246               35.370                   0.0   
       1989              44.704               33.100                   0.0   
       1990              43.965               33.665                   0.0   
       1991              44.308               32.750                   0.0   
       1992              44.575               32.397                   0.0   
       1993              44.928               32.779                   0.0   
       1994              44.649               32.664                   0.0   

             Nuclear (% sub energy)  Hydro (% sub energy)  \
Entity Year                                                 
Africa 1985                   0.664                 6.177   
       1986                   1.074                 6.240 

In [20]:
# Rename columns for ease of referencing
column_mappings = {
    'Coal (% electricity)': 'coal_production_perc',
    'Coal (% sub energy)': 'coal_consumption_perc',

    'Gas (% electricity)': 'gas_production_perc',
    'Gas (% sub energy)': 'gas_consumption_perc',

    'Hydro (% electricity)': 'hydro_production_perc',
    'Hydro (% sub energy)': 'hydro_consumption_perc',

    'Solar (% electricity)': 'solar_production_perc',
    'Solar (% sub energy)': 'solar_consumption_perc',

    'Wind (% electricity)': 'wind_production_perc',
    'Wind (% sub energy)': 'wind_consumption_perc',

    'Oil (% electricity)': 'oil_production_perc',
    'Oil (% sub energy)': 'oil_consumption_perc',

    'Nuclear (% electricity)': 'nuclear_production_perc',
    'Nuclear (% sub energy)': 'nuclear_consumption_perc',

    'Other renewables (% electricity)': 'other_renewable_production_perc',
    'Other renewables (% sub energy)': 'other_consumption_perc',
}

common_countries_by_year = common_countries_by_year.rename(columns=column_mappings).fillna(0)

In [21]:
condensed_df = common_countries_by_year.reset_index()
condensed_df.head()

,Entity,Year,coal_production_perc,gas_production_perc,hydro_production_perc,solar_production_perc,wind_production_perc,oil_production_perc,nuclear_production_perc,other_renewable_production_perc,oil_consumption_perc,coal_consumption_perc,solar_consumption_perc,nuclear_consumption_perc,hydro_consumption_perc,wind_consumption_perc,gas_consumption_perc,other_consumption_perc
0,Africa,1985,53.869485,10.783022,19.093587,0.0,0.0,13.242261,2.052236,0.103867,45.185,35.400,0.0,0.664,6.177,0.0,12.540,0.034
1,Africa,1986,52.600209,10.495367,18.935040,0.0,0.0,13.766315,3.259047,0.243975,43.475,35.372,0.0,1.074,6.240,0.0,13.759,0.080
2,Africa,1987,54.316611,11.269555,17.503940,0.0,0.0,13.802774,2.200460,0.234782,44.311,35.295,0.0,0.730,5.806,0.0,13.780,0.078
3,Africa,1988,51.625800,11.517103,18.149086,0.0,0.0,14.430585,3.553388,0.211991,43.246,35.370,0.0,1.158,5.915,0.0,14.242,0.069
4,Africa,1989,51.267484,12.046066,18.352067,0.0,0.0,14.132786,3.619461,0.201208,44.704,33.100,0.0,1.221,6.192,0.0,14.715,0.068


In [22]:
source_groups = condensed_df.groupby(condensed_df.columns.str.extract(r'^([^_]+)', expand=False), axis=1)

In [23]:
count = 0
test_series = []
for n, g in source_groups:
    if count == 2:
        prod_col, cons_col = g.columns;
        if 'production' not in prod_col:
            prod_col, cons_col = cons_col, prod_col
        g['source'] = n
        g = g.rename(columns={ prod_col : 'production', cons_col : 'consumption' })
        display(g)
        test_series = g.stack()
    count += 1

,production,consumption,source
0,53.869485,35.400,coal
1,52.600209,35.372,coal
2,54.316611,35.295,coal
3,51.625800,35.370,coal
4,51.267484,33.100,coal
...,...,...,...
2885,37.767379,29.059,coal
2886,36.688135,28.243,coal
2887,36.666964,27.853,coal
2888,36.562980,27.557,coal


In [24]:
def transformSourceGroup(group_df, group_name):
    if group_name in ['Entity', 'Year']:
        return group_df
    prod_col, cons_col = group_df.columns;
    if 'production' not in prod_col:
        prod_col, cons_col = cons_col, prod_col
    group_df['source'] = group_name
    group_df['footprint_ratio'] = (group_df[cons_col] / group_df[prod_col]).fillna(-1)
    return group_df.rename(columns={ prod_col : 'production', cons_col : 'consumption' })

In [25]:
transformed_df = source_groups.apply(lambda g: transformSourceGroup(g, g.name))
transformed_df

Entity  Year       coal                                           gas  \
      Entity  Year production consumption source footprint_ratio production   
0     Africa  1985  53.869485      35.400   coal        0.657144  10.783022   
1     Africa  1986  52.600209      35.372   coal        0.672469  10.495367   
2     Africa  1987  54.316611      35.295   coal        0.649801  11.269555   
3     Africa  1988  51.625800      35.370   coal        0.685123  11.517103   
4     Africa  1989  51.267484      33.100   coal        0.645633  12.046066   
...      ...   ...        ...         ...    ...             ...        ...   
2885   World  2015  37.767379      29.059   coal        0.769421  22.805036   
2886   World  2016  36.688135      28.243   coal        0.769813  23.056617   
2887   World  2017  36.666964      27.853   coal        0.759621  22.873988   
2888   World  2018  36.562980      27.557   coal        0.753686  22.470470   
2889   World  2019  35.064441      27.035   coal        0.771009  22.946552   

                                         ...  other                  \
     consumption source footprint_ratio  ... source footprint_ratio   
0         12.540    gas        1.162939  ...  other        0.327343   
1         13.759    gas        1.310959  ...  other        0.327902   
2         13.780    gas        1.222764  ...  other        0.332223   
3         14.242    gas        1.236596  ...  other        0.325486   
4         14.715    gas        1.221561  ...  other        0.337959   
...          ...    ...             ...  ...    ...             ...   
2885      23.051    gas        1.010786  ...  other        0.376787   
2886      23.270    gas        1.009255  ...  other        0.363896   
2887      23.502    gas        1.027455  ...  other        0.369636   
2888      24.063    gas        1.070872  ...  other        0.367696   
2889      24.226    gas        1.055758  ...  other        0.377378   

          solar                                          wind              \
     production consumption source footprint_ratio production consumption   
0      0.000000       0.000  solar       -1.000000   0.000000       0.000   
1      0.000000       0.000  solar       -1.000000   0.000000       0.000   
2      0.000000       0.000  solar       -1.000000   0.000000       0.000   
3      0.000000       0.000  solar       -1.000000   0.000000       0.000   
4      0.000000       0.000  solar       -1.000000   0.000000       0.000   
...         ...         ...    ...             ...        ...         ...   
2885   1.092341       0.431  solar        0.394565   3.554302       1.394   
2886   1.377463       0.540  solar        0.392025   4.008043       1.585   
2887   1.804265       0.718  solar        0.397946   4.619666       1.832   
2888   2.241461       0.905  solar        0.403754   4.953471       1.972   
2889   2.718259       1.105  solar        0.406510   5.470879       2.183   

                             
     source footprint_ratio  
0      wind       -1.000000  
1      wind       -1.000000  
2      wind       -1.000000  
3      wind       -1.000000  
4      wind       -1.000000  
...     ...             ...  
2885   wind        0.392201  
2886   wind        0.395455  
2887   wind        0.396566  
2888   wind        0.398105  
2889   wind        0.399022  

[2890 rows x 34 columns]

In [26]:
# Need to flatten the multi-index df (unfortantely current method involves creating a new df TWICE)
data_records = pd.DataFrame(transformed_df.to_records()).drop('index', axis=1).reindex()
data_records = data_records.rename(columns={"('Entity', 'Entity')": 'Entity', "('Year', 'Year')": 'Year'})
data_records.head()

,Entity,Year,"('coal', 'production')","('coal', 'consumption')","('coal', 'source')","('coal', 'footprint_ratio')","('gas', 'production')","('gas', 'consumption')","('gas', 'source')","('gas', 'footprint_ratio')",...,"('other', 'source')","('other', 'footprint_ratio')","('solar', 'production')","('solar', 'consumption')","('solar', 'source')","('solar', 'footprint_ratio')","('wind', 'production')","('wind', 'consumption')","('wind', 'source')","('wind', 'footprint_ratio')"
0,Africa,1985,53.869485,35.400,coal,0.657144,10.783022,12.540,gas,1.162939,...,other,0.327343,0.0,0.0,solar,-1.0,0.0,0.0,wind,-1.0
1,Africa,1986,52.600209,35.372,coal,0.672469,10.495367,13.759,gas,1.310959,...,other,0.327902,0.0,0.0,solar,-1.0,0.0,0.0,wind,-1.0
2,Africa,1987,54.316611,35.295,coal,0.649801,11.269555,13.780,gas,1.222764,...,other,0.332223,0.0,0.0,solar,-1.0,0.0,0.0,wind,-1.0
3,Africa,1988,51.625800,35.370,coal,0.685123,11.517103,14.242,gas,1.236596,...,other,0.325486,0.0,0.0,solar,-1.0,0.0,0.0,wind,-1.0
4,Africa,1989,51.267484,33.100,coal,0.645633,12.046066,14.715,gas,1.221561,...,other,0.337959,0.0,0.0,solar,-1.0,0.0,0.0,wind,-1.0


In [27]:
# Group/collect each field
production_cols = [x for x in data_records.columns if 'production' in x]
consumption_cols = [x for x in data_records.columns if 'consumption' in x]
footprint_cols = [x for x in data_records.columns if 'footprint' in x]

source_label_pattern = re.compile(r'\'([^\']+)\',')
source_labels = [re.findall(source_label_pattern, s)[0] for s in np.unique(np.array(production_cols))]

concat_production = pd.concat([data_records[['Entity', 'Year', x]] for x in production_cols], keys=source_labels)
concat_consumption = pd.concat([data_records[['Entity', 'Year', x]] for x in consumption_cols], keys=source_labels)
concat_footprint = pd.concat([data_records[['Entity', 'Year', x]] for x in footprint_cols], keys=source_labels)

concat_production['source'] = concat_production.index.get_level_values(0)
concat_consumption['source'] = concat_consumption.index.get_level_values(0)
concat_footprint['source'] = concat_footprint.index.get_level_values(0)

concat_production['production_perc'] = concat_production[production_cols].fillna(0).sum(axis=1)
concat_consumption['consumption_perc'] = concat_consumption[consumption_cols].fillna(0).sum(axis=1)
concat_footprint['footprint_ratio'] = concat_footprint[footprint_cols].fillna(0).sum(axis=1)

concat_production = concat_production.drop(production_cols, axis=1)
concat_consumption = concat_consumption.drop(consumption_cols, axis=1)
concat_footprint = concat_footprint.drop(footprint_cols, axis=1)

In [28]:
concat_production.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 23120 entries, ('coal', 0) to ('wind', 2889)
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Entity           23120 non-null  object 
 1   Year             23120 non-null  int64  
 2   source           23120 non-null  object 
 3   production_perc  23120 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 790.7+ KB


In [64]:
# Finally merge everything
concat_dfs = [concat_production, concat_consumption, concat_footprint]

agg_df = reduce(lambda left, right: pd.merge(left, right, on=['Entity', 'Year', 'source']), concat_dfs)

In [65]:
agg_df.head()

,Entity,Year,source,production_perc,consumption_perc,footprint_ratio
0,Africa,1985,coal,53.869485,35.400,0.657144
1,Africa,1986,coal,52.600209,35.372,0.672469
2,Africa,1987,coal,54.316611,35.295,0.649801
3,Africa,1988,coal,51.625800,35.370,0.685123
4,Africa,1989,coal,51.267484,33.100,0.645633


In [66]:
# Want to group by country to calc yearly change
country_groups = agg_df.groupby('Entity')

In [67]:
# Ensure only one datapoint per (Entity, Year) by averaging
agg_df_groups = agg_df.groupby(['Entity', 'Year']).mean(numeric_only=True)
agg_df_groups.head()

production_perc  consumption_perc  footprint_ratio
Entity Year                                                    
Africa 1985        12.393057              12.5         0.525834
       1986        12.412494              12.5         0.516062
       1987        12.416015              12.5         0.509816
       1988        12.435994              12.5         0.486979
       1989        12.452384              12.5         0.505380

In [68]:
# Calculate consecutive row differences within entity groups
target_cols = [c for c in agg_df_groups.columns if c != 'source']
agg_diff_df = agg_df_groups.groupby(level=0)[target_cols].diff().fillna(0) \
                                    .rename(columns={ c: f'{c}_diff' for c in target_cols})
agg_diff_df.head()

production_perc_diff  consumption_perc_diff  footprint_ratio_diff
Entity Year                                                                   
Africa 1985              0.000000                    0.0              0.000000
       1986              0.019437                    0.0             -0.009772
       1987              0.003521                    0.0             -0.006246
       1988              0.019979                    0.0             -0.022837
       1989              0.016390                    0.0              0.018401

In [69]:
# Merge diff values into original aggregated df
agg_dfs = [agg_df[['Entity', 'Year', 'source']], agg_df_groups, agg_diff_df]

transformed_df = reduce(lambda left, right: pd.merge(left, right, on=['Entity', 'Year']), agg_dfs)
transformed_df.head()

,Entity,Year,source,production_perc,consumption_perc,footprint_ratio,production_perc_diff,consumption_perc_diff,footprint_ratio_diff
0,Africa,1985,coal,12.393057,12.5,0.525834,0.0,0.0,0.0
1,Africa,1985,gas,12.393057,12.5,0.525834,0.0,0.0,0.0
2,Africa,1985,hydro,12.393057,12.5,0.525834,0.0,0.0,0.0
3,Africa,1985,nuclear,12.393057,12.5,0.525834,0.0,0.0,0.0
4,Africa,1985,oil,12.393057,12.5,0.525834,0.0,0.0,0.0


In [70]:
# Flatten df
transformed_df = transformed_df.reset_index() if 'index' not in transformed_df.columns else \
                                                    transformed_df.reset_index().drop('index', axis=1)
transformed_df.head()

,index,Entity,Year,source,production_perc,consumption_perc,footprint_ratio,production_perc_diff,consumption_perc_diff,footprint_ratio_diff
0,0,Africa,1985,coal,12.393057,12.5,0.525834,0.0,0.0,0.0
1,1,Africa,1985,gas,12.393057,12.5,0.525834,0.0,0.0,0.0
2,2,Africa,1985,hydro,12.393057,12.5,0.525834,0.0,0.0,0.0
3,3,Africa,1985,nuclear,12.393057,12.5,0.525834,0.0,0.0,0.0
4,4,Africa,1985,oil,12.393057,12.5,0.525834,0.0,0.0,0.0


In [71]:
# Replace infinite values with -1
transformed_df['footprint_ratio'] = transformed_df['footprint_ratio'].replace([np.inf, -np.inf], -1).fillna(-1)

In [72]:
transformed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23120 entries, 0 to 23119
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  23120 non-null  int64  
 1   Entity                 23120 non-null  object 
 2   Year                   23120 non-null  int64  
 3   source                 23120 non-null  object 
 4   production_perc        23120 non-null  float64
 5   consumption_perc       23120 non-null  float64
 6   footprint_ratio        23120 non-null  float64
 7   production_perc_diff   23120 non-null  float64
 8   consumption_perc_diff  23120 non-null  float64
 9   footprint_ratio_diff   23120 non-null  float64
dtypes: float64(6), int64(2), object(2)
memory usage: 1.8+ MB


In [73]:
# Output to file
file_path = '../data/generated_consumption_production_by_source.csv'
transformed_df.to_csv(file_path)

In [59]:
transformed_df['footprint_ratio'].describe()

count    1.293600e+04
mean              inf
std               NaN
min     -5.062298e-01
25%      6.660814e-01
50%      1.313594e+00
75%      2.525424e+00
max               inf
Name: footprint_ratio, dtype: float64